### Фаза 1: Ядро Индексации

In [22]:
import os
import time
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [2]:
load_dotenv()

True

In [3]:
file_path = os.path.join("data", "knowledge_base.txt")
loader = TextLoader(file_path, encoding='utf-8')

documents = loader.load()
documents

[Document(metadata={'source': 'data\\knowledge_base.txt'}, page_content='О компании "Космокофе"\n"Космокофе" — это передовая технологическая компания, основанная в 2025 году группой энтузиастов-инженеров и бариста. Наша главная миссия — сделать высококачественный кофе доступным для человечества за пределами Земли. Мы специализируемся на разработке, производстве и доставке кофейной продукции для экипажей орбитальных станций, лунных баз и долгосрочных космических миссий. Штаб-квартира компании расположена в инновационном центре "Сколково", а производственные мощности — на территории космодрома "Восточный".\nТехнология производства\nКлючевой инновацией "Космокофе" является технология "Крио-сублимационной обжарки" (КСО). В отличие от традиционных методов, обжарка зерен происходит в условиях глубокого вакуума и при криогенных температурах. Этот процесс позволяет сохранить до 98% ароматических масел и летучих соединений, которые обычно теряются при стандартной обжарке. После обжарки кофе гер

In [4]:
# Инициализируем сплиттер
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True
)

In [5]:
chunks = text_splitter.split_documents(documents)

print(f"Документ был разбит на {len(chunks)} чанков.")
print("\n--- Содержимое первого чанка ---")
print(chunks[0].page_content)
print("\n--- Метаданные первого чанка ---")
print(chunks[0].metadata)

Документ был разбит на 3 чанков.

--- Содержимое первого чанка ---
О компании "Космокофе"
"Космокофе" — это передовая технологическая компания, основанная в 2025 году группой энтузиастов-инженеров и бариста. Наша главная миссия — сделать высококачественный кофе доступным для человечества за пределами Земли. Мы специализируемся на разработке, производстве и доставке кофейной продукции для экипажей орбитальных станций, лунных баз и долгосрочных космических миссий. Штаб-квартира компании расположена в инновационном центре "Сколково", а производственные мощности — на территории космодрома "Восточный".
Технология производства

--- Метаданные первого чанка ---
{'source': 'data\\knowledge_base.txt', 'start_index': 0}


In [6]:
# Задаём имя модели, устройство и нормализацию эмбеддингов
model_name = 'cointegrated/rubert-tiny2'
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings': True}

In [7]:
# Инициализируем модель эмбеддингов
embeddings_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

C:\Users\79104\AppData\Local\Temp\ipykernel_23344\1420308842.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(


Пример эмбеддинга для простого предложения

In [8]:
example_embedding = embeddings_model.embed_query("Это тестовое предложение")

print(f"Размерность вектора (длина): {len(example_embedding)}")
print(f"Первые 5 значений вектора: {example_embedding[:5]}")

Размерность вектора (длина): 312
Первые 5 значений вектора: [-0.05698322132229805, -0.02797340415418148, -0.007402128539979458, -0.026605045422911644, -0.06908369809389114]


In [9]:
db_path = 'vector_store'

In [10]:
print('Создание векторной базы данных...')
vector_db = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings_model
)
vector_db.save_local(db_path)
print(f"Векторная база данных сохранена в папку: {db_path}")

Создание векторной базы данных...
Векторная база данных сохранена в папку: vector_store


### Фаза 2: Ядро Запроса-Ответа

In [11]:
# Загружаем векторную базу с диска
vector_db = FAISS.load_local(
    db_path,
    embeddings_model,
    allow_dangerous_deserialization=True
)
print("Векторная база данных успешно загружена.")

Векторная база данных успешно загружена.


Проверим, что база работает

In [12]:
query = "Сколько стоит эспрессо?"
results = vector_db.similarity_search(query)

print("\n--- Результат поиска по схожести ---")
print(results[0].page_content)


--- Результат поиска по схожести ---
"Лунное латте": Нежный кофейный напиток с добавлением сухого молока, обогащенного кальцием и минералами, идентичными по составу лунному реголиту. Идеально подходит для поддержания костной массы в условиях низкой гравитации. Цена за упаковку (20 саше) — 120 кредитов.
"Туманность Андромеды" (Кофе без кофеина): Для тех, кто хочет насладиться вкусом кофе без стимулирующего эффекта. Кофеин удаляется с помощью запатентованного метода экстракции углекислым газом. Стоимость — 110 кредитов за упаковку.
Доставка и логистика
Доставка осуществляется грузовыми кораблями "Прогресс" и частными транспортными компаниями, такими как "SpaceX" и "Blue Origin". Стандартное время доставки до Международной Космической Станции (МКС) составляет от 48 до 72 часов с момента запуска. Для лунных баз логистика занимает до 5 рабочих дней. Мы не осуществляем доставку на Марс в связи с текущими логистическими ограничениями.


In [13]:
# Превращаем нашу векторную базу в "ретривер"
retriever = vector_db.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 2}
)

In [14]:
query = "Какая миссия у компании Космокофе?"
retrieved_docs = retriever.invoke(query)

print("--- Результат работы ретривера ---")
for i, doc in enumerate(retrieved_docs):
    print(f"--- Документ {i+1} ---\n")
    print(doc.page_content)

--- Результат работы ретривера ---
--- Документ 1 ---

О компании "Космокофе"
"Космокофе" — это передовая технологическая компания, основанная в 2025 году группой энтузиастов-инженеров и бариста. Наша главная миссия — сделать высококачественный кофе доступным для человечества за пределами Земли. Мы специализируемся на разработке, производстве и доставке кофейной продукции для экипажей орбитальных станций, лунных баз и долгосрочных космических миссий. Штаб-квартира компании расположена в инновационном центре "Сколково", а производственные мощности — на территории космодрома "Восточный".
Технология производства
--- Документ 2 ---

"Лунное латте": Нежный кофейный напиток с добавлением сухого молока, обогащенного кальцием и минералами, идентичными по составу лунному реголиту. Идеально подходит для поддержания костной массы в условиях низкой гравитации. Цена за упаковку (20 саше) — 120 кредитов.
"Туманность Андромеды" (Кофе без кофеина): Для тех, кто хочет насладиться вкусом кофе без стимул

Настроим LLM

In [15]:
llm = ChatGoogleGenerativeAI(
    model='gemini-2.5-pro',
    temperature=0.1,
    convert_system_message_to_human=True
)

Проверим работу модели с простым запросом

In [ ]:
response = llm.invoke("Привет! Какой сегодня день?")
response.content

'Привет!\n\nЕсли у вас 20 октября 2025 года, то это **понедельник**.\n\nХорошего вам дня'

Теперь напишем шаблон промпта для взаимодействия пользователя и llm

In [17]:
RAG_PROMPT_TEMPLATE = """
Используй только следующий контекст, чтобы ответить на вопрос. Если ты не знаешь ответа на основе предоставленного контекста, просто скажи, что не знаешь. Не пытайся выдумать ответ. Отвечай на русском языке.

КОНТЕКСТ:
{context}

ВОПРОС:
{question}

ОТВЕТ:
"""

In [18]:
rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

Объединим retriever, llm и rag_prompt в один механизм

In [19]:
# Функция для форматирования найденных документов в единую строку
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

Для объединения компонентов в одну цепочку будем использовать синтаксис LCEL

In [20]:
rag_chain = (
    {'context': retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

Проверим работоспособность нашей цепочки

In [23]:
print("--- Тестирование RAG цепочки ---")

question1 = "Сколько стоит галактический эспрессо?"
print(f"Вопрос: {question1}")
answer1 = rag_chain.invoke(question1)
print(f"Ответ: {answer1}\n")

time.sleep(30) 
question2 = "Какова миссия компании?"
print(f"Вопрос: {question2}")
answer2 = rag_chain.invoke(question2)
print(f"Ответ: {answer2}\n")

time.sleep(30) 
question3 = "Какой кофе подают на Марсе?"
print(f"Вопрос: {question3}")
answer3 = rag_chain.invoke(question3)
print(f"Ответ: {answer3}")

--- Тестирование RAG цепочки ---
Вопрос: Сколько стоит галактический эспрессо?
Ответ: Стоимость одной стандартной упаковки (30 капсул) "Галактического эспрессо" составляет 100 галактических кредитов.

Вопрос: Какова миссия компании?
Ответ: Главная миссия компании — сделать высококачественный кофе доступным для человечества за пределами Земли.

Вопрос: Какой кофе подают на Марсе?
Ответ: На основе предоставленного контекста, я не знаю ответа на этот вопрос. В тексте указано, что доставка на Марс не осуществляется в связи с текущими логистическими ограничениями.


Модель отлично справилась со своей задачей